In [1]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpadhahyh8".


In [16]:
%%cuda
#include<assert.h>
#include<stdio.h>
#include<cuda.h>
#include<time.h>

void prodottoCPU(float *a, float *b, float *c, int n);
__global__ void prodottoGPU(float* a, float* b, float* c, float* u, int n);

int main(void) {
	float *a_h, *b_h, *c_h, *c_h2; // host data
	float *a_d, *b_d, *c_d; // device data
	float *u_d, *u_h;
	int N, nBytes, nBlockBytes, s_size;
	dim3 gridDim, blockDim;
	float elapsed_gpu, elapsed_cpu;
	cudaEvent_t start_gpu, stop_gpu, start_cpu, stop_cpu;

  N = 320000;
  blockDim.x = 32;

	// Determinazione esatta del numero di blocchi
	gridDim= N / blockDim.x + ((N%blockDim.x) == 0 ? 0 : 1);

	nBytes = N*sizeof(float);
	nBlockBytes = gridDim.x * sizeof(float);
	a_h = (float*)malloc(nBytes);
	b_h = (float*)malloc(nBytes);
	c_h = (float*)malloc(nBytes);
	c_h2 = (float*)malloc(nBytes);
	u_h = (float*)malloc(nBlockBytes);
	cudaMalloc((void **) &a_d, nBytes);
	cudaMalloc((void **) &b_d, nBytes);
	cudaMalloc((void **) &c_d, nBytes);
	cudaMalloc((void **) &u_d, nBlockBytes);

	// Inizializzo i dati
	// Inizializza la generazione random dei vettori utilizzando l'ora attuale del sistema
	srand((unsigned int) time(0));

	for (int i = 0; i < N; i++) {
		a_h[i] = rand()%5-2;
		b_h[i] = rand()%5-2;;
	}

	cudaMemcpy(a_d, a_h, nBytes, cudaMemcpyHostToDevice);
	cudaMemcpy(b_d, b_h, nBytes, cudaMemcpyHostToDevice);


	//azzeriamo il contenuto del vettore c
	memset(c_h, 0, nBytes);
	cudaMemset(c_d, 0, nBytes);

	s_size = blockDim.x * sizeof(float);

	//invocazione del kernel
	cudaEventCreate(&start_gpu);
	cudaEventCreate(&stop_gpu);
	cudaEventRecord(start_gpu);
	prodottoGPU<<<gridDim, blockDim, s_size>>>(a_d, b_d, c_d, u_d, N);
	cudaMemcpy(u_h, u_d, nBlockBytes, cudaMemcpyDeviceToHost);
	float sommaGPU = 0;
	for(int i = 0; i < gridDim.x; i++){
		sommaGPU+=u_h[i];
	}
	cudaEventRecord(stop_gpu);
	cudaEventSynchronize(stop_gpu);
	cudaEventElapsedTime(&elapsed_gpu, start_gpu, stop_gpu);
	cudaEventDestroy(start_gpu);
	cudaEventDestroy(stop_gpu);

	cudaEventCreate(&start_cpu);
	cudaEventCreate(&stop_cpu);
	cudaEventRecord(start_cpu);
	// calcolo somma seriale su CPU
	prodottoCPU(a_h, b_h, c_h2, N);
	float sommaCPU = 0;
	for(int i = 0; i < N; i++){
		sommaCPU += c_h2[i];
	}
	cudaEventRecord(stop_cpu);
	cudaEventSynchronize(stop_cpu);

	cudaEventElapsedTime(&elapsed_cpu, start_cpu, stop_cpu);
	cudaEventDestroy(start_cpu);
	cudaEventDestroy(stop_cpu);

	if (N<20) {
		for(int i = 0; i < N; i++)
			printf("a_h[%d]=%6.2f ",i, a_h[i]);
		printf("\n");
		for(int i = 0; i < N; i++)
			printf("b_h[%d]=%6.2f ",i, b_h[i]);
		printf("\n");
		for(int i = 0; i < gridDim.x; i++)
			printf("u_h[%d]=%6.2f ",i, u_h[i]);
		printf("\n");
	}

	printf("s_GPU = %6.2f\n", sommaGPU);
	printf("s_CPU = %6.2f\n", sommaCPU);
	printf("time_GPU = %f\n", elapsed_gpu);
	printf("time_CPU = %f\n", elapsed_cpu);
	assert(sommaGPU == sommaCPU);


	free(a_h); free(b_h); free(c_h); free(c_h2); free(u_h);
	cudaFree(a_d); cudaFree(b_d); cudaFree(c_d); cudaFree(u_d);
	return 0;
}

//Seriale
void prodottoCPU(float *a, float *b, float *c, int n) {
	for(int i = 0; i < n; i++)
		c[i] = a[i] * b[i];
}

//Parallelo
__global__ void prodottoGPU(float* a, float * b, float* c, float* u, int n) {
	extern __shared__ float s[];
	int index = threadIdx.x + blockIdx.x*blockDim.x;
	if(index < n)
		s[threadIdx.x] = a[index]*b[index];
	__syncthreads();

	// somma da ricombinare
	int dist = blockDim.x;

	for(int k = dist / 2; k > 0; k >>= 1){
		if(threadIdx.x < k){
			s[threadIdx.x] = s[threadIdx.x] + s[threadIdx.x + k];
		}
		__syncthreads();
	}

	// restituzione output
	if(threadIdx.x == 0){
		u[blockIdx.x] = s[0];
	}
}


s_GPU = 633.00
s_CPU = 633.00
time_GPU = 0.305824
time_CPU = 2.464768

